In [1]:
import os
import glob
import sys
import torch
import numpy as np
import torchvision
import tqdm
import random
import matplotlib.pyplot as plt
import PIL
import torchvision.models as models

In [2]:
from torch import nn
from torch.nn import functional as F
from torchvision.io import read_image
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from tqdm.auto import tqdm
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [4]:
transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.5,0.5,0.5,0.3),
        transforms.RandomRotation(45, resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
    ])

C:\Users\Dmitry\anaconda3\envs\cuDNN\lib\site-packages\torchvision\transforms\transforms.py:1292: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed 0.14. Please use 'interpolation' instead.
  warnings.warn(


In [5]:
orig_set = torchvision.datasets.ImageFolder("C:\\Users\\Dmitry\\Desktop\\CNN2\\images\\Images", transform=transform)
train_size = int(0.33 * len(orig_set))
test_size = len(orig_set) - train_size
test_set, train_set = torch.utils.data.random_split(orig_set, [train_size, test_size], generator=torch.Generator().manual_seed(42))

In [6]:
print(f'Длина train_set: {len(train_set)}\nДлина test_set: {len(test_set)}')

Длина train_set: 13789
Длина test_set: 6791


In [7]:
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size = 32, shuffle = True, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(test_set, batch_size = 32, shuffle = False, num_workers=4)

In [8]:
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 120)
criterion = nn.CrossEntropyLoss()
n_epochs = 10
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.01)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
def accuracy(testloader, net, device="cuda:0"):
    correct = 0
    total = 0
    net = net.eval()
    with tqdm(total=len(testloader), file=sys.stdout) as prbar:
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prbar.update(1)
    print(f'correct: {correct}, total: {total}')
    accuracy = 100 * correct // total
    return accuracy

In [11]:
def train(trainloader, testloader, net, epochs, optimizer, criterion, device="cuda:0", scheduler = None):
    for epoch in range(epochs):
        print(f"{epoch} Эпоха на обучающей выборке ")
        net = net.train()
        correct = 0
        total = 0
        with tqdm(total=len(trainloader), file=sys.stdout) as prbar:
            for inputs, labels in trainloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                prbar.update(1)
        with torch.no_grad():
            accuracy_train = 100 * correct // total
            print(f'Точность на обучающей выборке: {accuracy_train}%')
            print(f"{epoch} Эпоха на тестовой выборке")
            print(f'Точность на валидационной выборке: {accuracy(testloader, net, device="cuda:0")}%')
        pass

Нейросеть №1

In [12]:
train(train_dataloader, val_dataloader, resnet18, n_epochs, optimizer, criterion, device)

0 Эпоха на обучающей выборке 


  0%|          | 0/431 [00:00<?, ?it/s]

Точность на обучающей выборке: 14%
0 Эпоха на тестовой выборке


  0%|          | 0/213 [00:00<?, ?it/s]

correct: 2119, total: 6791
Точность на валидационной выборке: 31%
1 Эпоха на обучающей выборке 


...

correct: 3625, total: 6791
Точность на валидационной выборке: 53%
9 Эпоха на обучающей выборке 


  0%|          | 0/431 [00:00<?, ?it/s]

Точность на обучающей выборке: 56%
9 Эпоха на тестовой выборке


  0%|          | 0/213 [00:00<?, ?it/s]

correct: 3602, total: 6791
Точность на валидационной выборке: 53%


In [13]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.5,0.5,0.5,0.3),
        transforms.RandomRotation(45, resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
    ])

In [14]:
orig_set = torchvision.datasets.ImageFolder("C:\\Users\\Dmitry\\Desktop\\CNN2\\images\\Images", transform=transform)
train_size = int(0.33 * len(orig_set))
test_size = len(orig_set) - train_size
test_set, train_set = torch.utils.data.random_split(orig_set, [train_size, test_size], generator=torch.Generator().manual_seed(42))

In [21]:
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size = 32, shuffle = True, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(test_set, batch_size = 32, shuffle = False, num_workers=4)

Нейросеть №2

In [22]:
resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 120)
optimizer = torch.optim.SGD(resnet50.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, 'min')
resnet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
train(train_dataloader, val_dataloader, resnet50, n_epochs, optimizer, criterion, device, scheduler)

0 Эпоха на обучающей выборке 


  0%|          | 0/431 [00:00<?, ?it/s]

Точность на обучающей выборке: 25%
0 Эпоха на тестовой выборке


  0%|          | 0/213 [00:00<?, ?it/s]

correct: 2981, total: 6791
Точность на валидационной выборке: 43%
1 Эпоха на обучающей выборке 


...

correct: 3579, total: 6791
Точность на валидационной выборке: 52%
9 Эпоха на обучающей выборке 


  0%|          | 0/431 [00:00<?, ?it/s]

Точность на обучающей выборке: 65%
9 Эпоха на тестовой выборке


  0%|          | 0/213 [00:00<?, ?it/s]

correct: 3788, total: 6791
Точность на валидационной выборке: 55%


In [24]:
PATH = 'C:\\Users\\Dmitry\\Desktop\\CNN2\\resnet50.pth'
torch.save(resnet50.state_dict(), PATH)
PATH = 'C:\\Users\\Dmitry\\Desktop\\CNN2\\resnet18.pth'
torch.save(resnet18.state_dict(), PATH)